### Task 1. 

#### MapReduce

Use MapReduce to calculate the average housing price in each zip code and filter out records in which gross sqft is less than 500 and sale price is less than 100000. Round the average housing price to 2 decimals, and sort the output by housing price ascending.

In [36]:
import pandas as pd
import csv
from functools import reduce
from multiprocessing import Pool

In [37]:
import wget
#url = 'https://github.com/CUSP2022ADS/Data/raw/main/housingSalseSample.csv'
#wget.download(url)

In [38]:
import pandas as pd
housingSalseSample = pd.read_csv('housingSalseSample.csv')
housingSalseSample['AVG_Price'] = 0
housingSalseSample.to_csv('housingSalseSample.csv')
housingSalseSample.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,BOROUGH,BLOCK,LOT,BUILDING CLASS CATEGORY,ADDRESS,ZIP CODE,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE,SALE DATE,AVG_Price
0,0,0,0,1,1170,1010,17 CONDOPS,"200 WEST 79TH STREET, 7P",10024.0,0.0,1975.0,551000,2011-07-28,0
1,1,1,1,4,1275,45,10 COOPS - ELEVATOR APARTMENTS,"35-35 75TH STREET, 304",11372.0,0.0,1941.0,338000,2011-05-20,0
2,2,2,2,3,5746,58,03 THREE FAMILY DWELLINGS,1249 65TH STREET,11219.0,3861.0,1910.0,1250000,2014-12-04,0
3,3,3,3,4,2086,50,10 COOPS - ELEVATOR APARTMENTS,"97-07 63 ROAD, 8F",11374.0,0.0,1955.0,253000,2016-03-09,0
4,4,4,4,4,5028,7,10 COOPS - ELEVATOR APARTMENTS,"38-15 149TH STREET, 1F",11354.0,0.0,1955.0,0,2016-12-30,0


In [60]:
# code here
def returnInfo(record): #extract necessary fields from a data record (raw)
    return(record['ZIP CODE'],record['GROSS SQUARE FEET'],record['SALE PRICE'])
    
def priceList(result,record):
    zipcode, price = record[0], record[2]
    result[zipcode] = result.get(zipcode,[]) + [price]
    return result

def averageprice(record):
    t_p = 0
    zipcode, price = record[0], record[1]
    for i in record[1]:
        t_p += float(i)
    return [zipcode, t_p/len(price)]

def filt(record):
    sqft = record[1]
    if sqft != '':
        sale_price = float(record[2])
        sqft = float(sqft)
        if sqft > 500 and sale_price > 100000:
            return True
        else:
            return False

In [61]:
with open('housingSalseSample.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    output = list(map(averageprice,
                       list(reduce(priceList,
                                   list(filter(filt, 
                                               map(returnInfo,reader))),
                                                {}).items())))
output[:3]

[['11219.0', 1327215.7661290322],
 ['10472.0', 795404.6736842105],
 ['11214.0', 1035225.83125]]

In [62]:
pd.DataFrame(output,columns=['zipcode','price']).head()

,zipcode,price
0,11219.0,1.327216e+06
1,10472.0,7.954047e+05
2,11214.0,1.035226e+06
3,11385.0,8.721293e+05
4,11235.0,9.822538e+05


In [63]:
'''grading function
please pass your output into this function and upload the generated csv file 
together with your notebook to your GitHub repo'''

def grading(output):
    import pandas as pd
    pd.DataFrame(output,columns=['zipcode','price']).to_csv('output1.csv',index=False)

grading(output)

### Task 2. 
#### multiprocessing and Chunk

In [188]:
#!wget -O ADSSession2Task2Data.csv https://raw.githubusercontent.com/CUSP2022ADS/Data/main/ADSSession2Task2Data.csv
#url = 'https://raw.githubusercontent.com/CUSP2022ADS/Data/main/ADSSession2Task2Data.csv'
#wget.download(url)

'ADSSession2Task2Data.csv'

Apply the chunk and MapReduce method to sum up each type's value in the synthetic dataset—only keep records which $Select$ value is 1, and read 1000 lines in each chunk. Use multiprocessing package to deploy the task to multi-cores. The output should only have two columns: type and total value.

Hint: filter function is not available in multiprocessing, implement an if-else function in the map or reduce step.

In [32]:
task2 = pd.read_csv('ADSSession2Task2Data.csv')
task2.head()

,Type,Value,Select
0,0,0.1488486295446153,0
1,3,0.2198529412734188,1
2,7,0.15871379686112896,0
3,0,0.9980337347337275,1
4,3,0.4923200916998063,0


In [33]:
from ipynb.fs.full.function import returnInfo
from ipynb.fs.full.function import TypeList
from ipynb.fs.full.function import sumvalue
result = []
with open('ADSSession2Task2Data.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    chunk = []
    chunksize = 1000
    pool = Pool(8)
    for i, line in enumerate(reader):
        
        if (i % chunksize == 0 and i > 0):
            # MapReduce function for rows in one chunk
            output = list(pool.map(sumvalue,
                         list(reduce(TypeList,
                             list(pool.map(returnInfo,chunk)
                            ),
                         {}).items())))
            result += output
            chunk = []
        chunk.append(line)
    
    output = list(pool.map(sumvalue,
                         list(reduce(TypeList,
                             list(pool.map(returnInfo,chunk)
                            ),
                         {}).items())))
    result += output
    pool.close()
    

In [34]:
def resultList(result,record):
    Type, value = record[0],record[1]
    result[Type] = result.get(Type,[]) + [value]
    return result

def sumresult(record):
    Type, value = record[0],record[1]
    return [Type, sum(value)]

output = list(map(sumresult,list(reduce(resultList,result,{}).items())))
output[:3]

[['3', 294.32549925839936], ['0', 306.7099300824752], ['1', 316.7932728146583]]

In [35]:
#from ipynb.fs.full.function import square
#with Pool(8) as p:
#    result = list(p.map(square,range(100)))
#    p.close()

In [31]:
'''grading function
please pass your output into this function and upload the generated csv file 
together with your notebook to your GitHub repo'''

def grading(output):
    import pandas as pd
    pd.DataFrame(output,columns=['Type','Value']).to_csv('output2.csv',index=False)

grading(output)